In [1]:
import torch
import numpy as np

In [4]:
t1 = torch.load("results_cov/attn/new_model/weights.pth")
t1

{'channel_1_attn_map': [tensor([[[0.2766, 0.2704, 0.1853, 0.1099, 0.0767, 0.0811],
           [0.2864, 0.2741, 0.1828, 0.1048, 0.0734, 0.0785],
           [0.2991, 0.2801, 0.1816, 0.0983, 0.0677, 0.0734],
           [0.3006, 0.2801, 0.1837, 0.0978, 0.0660, 0.0719],
           [0.2978, 0.2785, 0.1876, 0.0987, 0.0658, 0.0716],
           [0.2796, 0.2665, 0.1920, 0.1074, 0.0750, 0.0795]],
  
          [[0.2752, 0.2653, 0.1760, 0.1106, 0.0851, 0.0878],
           [0.2840, 0.2680, 0.1730, 0.1071, 0.0823, 0.0856],
           [0.2952, 0.2725, 0.1711, 0.1026, 0.0774, 0.0813],
           [0.3001, 0.2744, 0.1722, 0.1007, 0.0742, 0.0784],
           [0.2922, 0.2695, 0.1762, 0.1047, 0.0767, 0.0807],
           [0.2733, 0.2571, 0.1806, 0.1143, 0.0858, 0.0888]],
  
          [[0.2652, 0.2574, 0.1789, 0.1158, 0.0899, 0.0927],
           [0.2742, 0.2606, 0.1759, 0.1120, 0.0869, 0.0903],
           [0.2854, 0.2652, 0.1740, 0.1074, 0.0819, 0.0860],
           [0.2898, 0.2666, 0.1749, 0.1059, 0.0792, 0.0

In [14]:
final_attn_map = None

for i in range(1, 5):
    attn_weights = t1[f'channel_{i}_attn_map']
    weights = t1[f'channel_{i}_weights']
    if final_attn_map is None:
        final_attn_map = (attn_weights[0] + attn_weights[1]) * t1['final_weights'][i-1]
    else:
        final_attn_map += (attn_weights[0] + attn_weights[1]) * t1['final_weights'][i-1]


In [15]:
final_attn_map.shape

torch.Size([29800, 6, 6])

In [56]:
interpret = torch.sum(final_attn_map[56 * 149 : 57 * 149], dim=1)

In [57]:
interpret.shape

torch.Size([149, 6])

In [64]:
values, indices = torch.topk(interpret, k=7, dim=0)
values.shape, indices.shape

(torch.Size([7, 6]), torch.Size([7, 6]))

In [65]:
indices_np = indices.cpu().detach().numpy()
indices_np

array([[133,  33, 133,  91, 100,  93],
       [ 33,  32, 121,  46, 148,  92],
       [121,  80,   5,  93,  54, 100],
       [ 32,  34, 122,  47, 101, 105],
       [ 44,  23,   7,  92,   5, 148],
       [ 64,  60,   6,  26, 144,  73],
       [ 34,  74, 129,  30, 138,  54]])

In [49]:
epitopes = []
with open('data/cov_epitopes/epitopes_sorted.txt', 'r') as f:
    for line in f:
        epitopes.append(int(line))

In [50]:
# epitopes = np.tile(epitopes, (6, 1)).T
epitopes = np.asarray(epitopes).reshape(-1, 1)
epitopes.shape

(149, 1)

In [66]:
epitopes[indices_np].squeeze()

array([[ 979,  153,  979,  452,  468,  458],
       [ 153,  152,  716,  181, 1229,  453],
       [ 716,  367,   18,  458,  208,  468],
       [ 152,  154,  793,  182,  476,  487],
       [ 179,  143,   20,  453,   18, 1229],
       [ 248,  222,   19,  146, 1142,  257],
       [ 154,  258,  852,  150, 1027,  208]])

In [8]:
list(t1.keys())

['channel_1_attn_map',
 'channel_2_attn_map',
 'channel_3_attn_map',
 'channel_4_attn_map',
 'channel_1_weights',
 'channel_2_weights',
 'channel_3_weights',
 'channel_4_weights',
 'final_weights']

In [7]:
t1['final_weights']

Parameter containing:
tensor([1.0183, 0.8773, 1.0966, 1.0100], device='cuda:0', requires_grad=True)